In [2]:
import numpy as np
from ppe_tools import Ensemble,Member,ParamInfo
from ppe_tools.utils import get_default, parse_val
import xarray as xr
import os
import pandas as pd
import glob

In [3]:
csv='lhc_220926.csv'

In [4]:
data_url = '"https://docs.google.com/spreadsheets/d/e/2PACX-1vQLjMUT7sgunI4XkbGWq_RrjrLhR8-N0RiojM7h3lbSsN5-jgKIFyiJtPJeQkTEf2TL4ouLu_dqmEkg/pub?gid=737792973&single=true&output=csv"'
cmd = 'curl -L '+data_url+' > '+csv # need to add -L option to force redirects
os.system(cmd)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   460    0   460    0     0   1367      0 --:--:-- --:--:-- --:--:--  1369
100  5444    0  5444    0     0   9212      0 --:--:-- --:--:-- --:--:-- 25679


0

In [5]:
df = pd.read_csv(csv)
params=df['param'].values

In [7]:
kcns=['kc_nonmyc',
        'kn_nonmyc',
        'akc_active',
        'akn_active',
        'ekc_active',
        'ekn_active']
kcns

['kc_nonmyc',
 'kn_nonmyc',
 'akc_active',
 'akn_active',
 'ekc_active',
 'ekn_active']

In [8]:
lhcs = {}
for p in params:
    ix       = df['param']==p
    minval   = df['min'][ix].values[0]
    maxval   = df['max'][ix].values[0]
    pftmin   = df['pft_mins'][ix].values[0]
    pftmax   = df['pft_maxs'][ix].values[0]
    thisloc  = df['loc'][ix].values[0]
    if p=='KCN':
        flag='KCN'
    else:
        flag=''
    
    needs_pft = (minval=='pft')
    if needs_pft:
        thismin = np.fromstring(pftmin, dtype='float', sep=',')
    elif 'percent' in minval:
        thismin = minval
    else:
        thismin = np.array(float(minval))

    needs_pft = (maxval=='pft')
    if needs_pft:
        thismax = np.fromstring(pftmax, dtype='float', sep=',')
    elif 'percent' in maxval:
        thismax = maxval
    else:
        thismax = np.array(float(maxval))

    if p!='KCN':
        lhcs[p]={'min':thismin,'max':thismax,'loc':thisloc,'flagged':[]}
    else:
        flagged={}
        for kcn in kcns:
            flagged[kcn]={'min':thismin,'max':thismax,'loc':thisloc}
        lhcs[p]={'min':[],'max':[],'loc':thisloc,'flagged':flagged}

In [9]:
#instantiate the Ensemble object
basefile = '/glade/scratch/djk2120/PPEn11/paramfiles/OAAT0000.nc'
pdir = '/glade/scratch/djk2120/PPEn11/paramfiles/'
ndir = '/glade/scratch/djk2120/PPEn11/namelist_mods/'
x    = Ensemble(basefile,pdir,ndir)

#add the new LHCs
prefix = 'LHC'
nextnum=1
n_samples=500
x.add_lhcs(lhcs,prefix,nextnum,n_samples)

In [10]:
x.write(default_key='LHC0000',lhcfile='/glade/scratch/djk2120/PPEn11/lhc220926.txt')